# Raman data processing
#### Calculating area under peak from Raman files and processing to gas mol%

In [1]:
# ---------------- processing fluid inclusion Raman data (Raman spectroscopy)


# --- import modules

import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import plotly.express as px

import warnings

from module_peak_area import peak_area

# --- set working directories


base_dir = ".DATA/Raman/processed/"


In [24]:
# --- look into working directory and create a list with selected files - .txt
os.chdir(base_dir)

FI_files = glob.glob('*-v*.txt')


# print(FI_files)
# print(len(FI_files))

df_Raman = pd.DataFrame()


np.seterr(divide='ignore', invalid='ignore')
warnings.filterwarnings('ignore')


for file in FI_files:
#     print(file)
    current_file = pd.read_csv(base_dir+file,
                    encoding = "ANSI", sep = '\t', names=('x','y'), comment="#")

    # --- add file information
    current_inclusion = file.replace(".txt", "")
    
    
    data = peak_area(current_file,current_inclusion)
    a = pd.DataFrame(data).T
    
    a.insert(0, 'Sample', current_inclusion)
    
    
    df_Raman = pd.concat([df_Raman,a], axis=0, ignore_index=True)
    
df_Raman = df_Raman.fillna(0)

df_Raman

the CH4 peak range was not measured for sample bu02aw-a-ff1-fi01-v_1800 (450-850nm)_532nm_edge_100%_x100_vis_lwd_h50um_30sx2_01
the CH4 peak range was not measured for sample bu02aw-a-ff1-fi01-v_1800 (450-850nm)_532nm_edge_100%_x100_vis_lwd_h50um_30sx2_edited
the CH4 peak range was not measured for sample bu02aw-a-ff1-fi02-v_1800 (450-850nm)_532nm_edge_100%_x100_vis_lwd_h100um_30sx2_01
the CH4 peak range was not measured for sample bu02aw-a-ff1-fi03-v_1800 (450-850nm)_532nm_edge_100%_x100_vis_lwd_h100um_30sx2_01
the CH4 peak range was not measured for sample bu02aw-a-ff1-fi04-v_1800 (450-850nm)_532nm_edge_100%_x100_vis_lwd_h100um_30sx2_01
the CH4 peak range was not measured for sample bu02aw-a-ff1-fi05-v_1800 (450-850nm)_532nm_edge_100%_x100_vis_lwd_h50um_30sx2_01
the CH4 peak range was not measured for sample bu04w-a-ff3-fi02-v_1800 (450-850nm)_633nm_edge_100%_x100_vis_lwd_h50um_45sx2_01
the CH4 peak range was not measured for sample bu04w-a-ff3-fi03-v_1800 (450-850nm)_633nm_edge_100%

,Sample,area_CO2_v1,area_CO2_v2,area_N2,area_CH4
0,bu02aw-a-ff1-fi01-v_150 (500nm)_532nm_edge_100...,5428.358398,10264.656514,623.942231,32.136546
1,bu02aw-a-ff1-fi01-v_1800 (450-850nm)_532nm_edg...,785.490944,1590.678348,89.457797,0.000000
2,bu02aw-a-ff1-fi01-v_1800 (450-850nm)_532nm_edg...,785.489830,1467.048266,89.458660,0.000000
3,bu02aw-a-ff1-fi02-v_150 (500nm)_532nm_edge_100...,60698.129236,85789.711595,6976.669811,8.638066
4,bu02aw-a-ff1-fi02-v_1800 (450-850nm)_532nm_edg...,8257.747429,13629.626146,1103.262392,0.000000
...,...,...,...,...,...
302,bu32-C-FF1-FI06-v,8324.778585,16930.583122,1628.386021,99.125348
303,bu32-C-FF1-FI07-v,12035.956410,19809.951464,2003.745265,726.488774
304,bu32-C-FF1-FI08-v,16063.492408,25221.148116,2091.070056,185.872859
305,bu32-C-FF1-FI09-v,14867.137116,22373.159783,2335.339607,340.844722


In [25]:
Raman_cross_section = {'SO2' : 5.3,
                       '12CO2_v1' : 1,
                       '12CO2_2v2' : 1.5,
                       '13CO2_2v2' :1.5,
                       'O2' : 1.2,
                       'CO' : 0.9,
                       'N2' : 1,
                       'H2S' : 6.4,
                       'CH4' : 7.5,
                       'NH3' : 5,
                       'H2' : 2.3}


# print(df_Raman)

df_V_mol = df_Raman[['Sample']].copy()

df_V_mol


df_V_mol['CO2_v1_mol'] = df_Raman['area_CO2_v1'] / Raman_cross_section['12CO2_v1']
df_V_mol['CO2_v2_mol'] = df_Raman['area_CO2_v2'] / Raman_cross_section['12CO2_2v2']
df_V_mol['N2_mol'] = df_Raman['area_N2'] / Raman_cross_section['N2']
df_V_mol['CH4_mol'] = df_Raman['area_CH4'] / Raman_cross_section['CH4']



df_V_mol['mol_sum'] = df_V_mol.sum(axis=1)

df_Raman['X_CO2(mol%)'] = round(((df_V_mol['CO2_v1_mol']/df_V_mol['mol_sum']) + (df_V_mol['CO2_v2_mol']/df_V_mol['mol_sum'])),3)
df_Raman['X_N2(mol%)'] = round((df_V_mol['N2_mol']/df_V_mol['mol_sum']),3)
df_Raman['X_CH4(mol%)'] = round((df_V_mol['CH4_mol']/df_V_mol['mol_sum']),3)



df_Raman[['sample','piece','field','analysis','rest']
        ] = df_Raman["Sample"].str.split(pat='-', 
                                         n=4,  
                                         expand=True).replace(('w','bu'), 
                                                              ('','BU18FA'), 
                                                              regex=True)
df_Raman['field'] = df_Raman['field'].astype(str).replace(('FF'),('ff'),regex=True)
df_Raman['analysis'] = df_Raman['analysis'].astype(str).replace(('FI'),('fi'),regex=True)

df_Raman['analysis'] = df_Raman['analysis'].astype(str).replace(('10','11','12'),('010','011','012'),regex=True)


df_FI_Raman = df_Raman[['sample', 'piece', 'field', 'analysis',
                     'X_CO2(mol%)', 'X_N2(mol%)', 'X_CH4(mol%)',
                     'area_CO2_v1', 'area_CO2_v2', 'area_N2', 'area_CH4', 'rest']]

df_FI_Raman['piece'] = df_FI_Raman['piece'].str.upper()

df_FI_Raman.to_csv('Buranga_FI_Raman_V_composition.csv',index=False)
df_FI_Raman

,sample,piece,field,analysis,X_CO2(mol%),X_N2(mol%),X_CH4(mol%),area_CO2_v1,area_CO2_v2,area_N2,area_CH4,rest
0,BU18FA02a,A,ff1,fi01,0.951,0.048,0.000,5428.358398,10264.656514,623.942231,32.136546,v_150 (500nm)_532nm_edge_100%_x100_vis_ld_h30u...
1,BU18FA02a,A,ff1,fi01,0.954,0.046,0.000,785.490944,1590.678348,89.457797,0.000000,v_1800 (450-850nm)_532nm_edge_100%_x100_vis_ld...
2,BU18FA02a,A,ff1,fi01,0.952,0.048,0.000,785.489830,1467.048266,89.458660,0.000000,v_1800 (450-850nm)_532nm_edge_100%_x100_vis_ld...
3,BU18FA02a,A,ff1,fi02,0.944,0.056,0.000,60698.129236,85789.711595,6976.669811,8.638066,v_150 (500nm)_532nm_edge_100%_x100_vis_ld_h50u...
4,BU18FA02a,A,ff1,fi02,0.940,0.060,0.000,8257.747429,13629.626146,1103.262392,0.000000,v_1800 (450-850nm)_532nm_edge_100%_x100_vis_ld...
...,...,...,...,...,...,...,...,...,...,...,...,...
302,BU18FA32,C,ff1,fi06,0.923,0.077,0.001,8324.778585,16930.583122,1628.386021,99.125348,v
303,BU18FA32,C,ff1,fi07,0.923,0.073,0.004,12035.956410,19809.951464,2003.745265,726.488774,v
304,BU18FA32,C,ff1,fi08,0.940,0.060,0.001,16063.492408,25221.148116,2091.070056,185.872859,v
305,BU18FA32,C,ff1,fi09,0.926,0.073,0.001,14867.137116,22373.159783,2335.339607,340.844722,v


In [ ]:
fig = px.scatter_ternary(df_vapor, a ='X_CO2', b ='X_N2', c = 'X_CH4')
#                  symbol = "Sample", color="Host", 
#                  hover_data=["Sample","FI","Phases","Host"],
#                  color_discrete_map = colors,
#                 )

fig.update_layout(plot_bgcolor="white",template="simple_white")

fig.show()

In [29]:
laser = input("Insert laser wavelenght (nm):")
laser = int(laser)

Insert laser wavelenght (nm):532


In [30]:
v0 = 10**7 / laser #convert laser wavelength from nm to cm-1

vi = 2917

h = 6.626*(10**(-27))

c = 2.998*(10**(10))

k = 1.381*(10**(-16))

T = 273.15 + 20

suma = 8.63

In [31]:
sigma = suma/(((v0-vi)**(-4))/((v0-2331)**(-4)))*(1-(np.exp((-h*c*vi)/(k*T))))

#formula from Burke (2001) Lithos

sigma = round(sigma,1)

sigma

7.5